<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/S1_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install vllm -q
!pip install huggingface_hub -q
!pip install colab_env -q

!pip uninstall -y pyzmq -q
!pip install --no-cache-dir pyzmq -q
!pip install --upgrade vllm -q

!pip install --upgrade torch torchvision torchaudio -q
!pip install --upgrade transformers -q
!pip install accelerate -q
!pip install -U flash-attn --no-build-isolation --quiet

In [ ]:
import colab_env
import os


access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
from vllm import LLM, SamplingParams
import os

# Pass the context to the LLM constructor
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2", trust_remote_code=True, hf_token=access_token_write)

In [ ]:
prompts = [
    "The capital of France is",
    "Explain the theory of relativity in simple terms."
]
sampling_params = SamplingParams(max_tokens=50)

outputs = llm.generate(prompts, sampling_params)
print('\n')

In [5]:
print('\n')
for prompt, output in zip(prompts, outputs):
    print("-" * 20)
    print(f"Prompt: {prompt}")
    print(f"Output: {output.outputs[0].text}")
    print("-" * 20)
    print('\n')



--------------------
Prompt: The capital of France is
Output:  an exceptional city: Paris, the city of art and knowledge. Around 13 million tourists visit this part of Europe every year. Undoubtedly, Paris is the most popular city in the world, and it is not surprising, because it has everything
--------------------


--------------------
Prompt: Explain the theory of relativity in simple terms.
Output: 

The theory of relativity, proposed by Albert Einstein in the early 1900s, is a scientific explanation of how the physical world works at large scales and high speeds. It consists of two parts: the special theory of rel
--------------------




In [ ]:
import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
device= device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Pass the context to the LLM constructor
model = LLM(
    "simplescaling/s1.1-7B",
    tensor_parallel_size=1,
    device=device,
)

tok = AutoTokenizer.from_pretrained("simplescaling/s1-32B")
stop_token_ids = tok("<|im_end|>")["input_ids"]

sampling_params = SamplingParams(
    max_tokens=32768,
    min_tokens=0,
    stop_token_ids=stop_token_ids,
)

In [ ]:
prompt = "Count all occurrences of the letter 'r' in the word \"raspberry.\""
prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant. Your task is to accurately count the number of times a specific letter appears in a given word.\n<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"

o = model.generate(prompt, sampling_params=sampling_params)

In [3]:
print(o[0].outputs[0].text)

think
Okay, so I need to count how many times the letter 'r' appears in the word "raspberry." Let me try to think step by step. First, maybe I should write down the word and then go through each letter one by one, checking if it's an 'r'. The word is "raspberry."

Let me break it down:

1. The first letter is 'r', so that's one 'r'.
2. Next is 'a', no 'r'.
3. 's' – nope.
4. 'p' – not an 'r'.
5. 'b' – also not.
6. 'e' – nope.
7. 'r' – another 'r'! That's two now.
8. 'b' – no.
9. 'e' – nothing.
10. 'r' – there's a third 'r' at the end. So three 'r's?

Wait, let me double-check. Let's list each letter again: r-a-s-p-b-e-r-r-y. Yes, so:

First letter: r
Third letter: r
Last two letters: r and r

So yeah, three 'r's. I think that’s correct. But maybe I should think of another way to ensure I didn't skip any. What if I count each occurrence? Or maybe use a different approach. Alternatively, perhaps use a systematic check.

Breaking the word: "raspberry" into syllables or sounds: r-a-sp-berry